In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
import keras.layers as kl
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

# The RNN model implementation 

## Loading and splitting the data

In [ ]:
#loading the data
X = np.load("X_data.npy")
Y = np.load("Y_data.npy")